In [1]:
!pip install spacy_transformers
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: spacy-legacy
    Found existing installation: spacy-legacy 3.0.10
    Uninstalling spacy-legacy-3.0.10:
      Successfully uninstalled spacy-legacy-3.0.10
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: pathy
    Found existing installation: pathy 0.6.2
    Uninstalling pathy-0.6.2:
      Successfully uninstalled pathy-0.6.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.3.1
    Uninstalling spacy-3.3.1:
      Successfull

In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm  
import json

2023-02-08 19:47:02.809864: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 19:47:02.810744: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 19:47:02.811726: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 19:47:02.812431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 19:47:02.813111: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [3]:
spacy.__version__

'3.5.0'

In [4]:
cv_data = json.load(open('/kaggle/input/resumeparsing/data/training/train_data.json'))

In [5]:
len(cv_data)

200

In [6]:
!python -m spacy init fill-config /kaggle/input/resumeparsing/data/training/base_config.cfg config.cfg

2023-02-08 19:47:24.734314: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 19:47:24.735218: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 19:47:24.736219: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 19:47:24.736977: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 19:47:24.737751: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [7]:
cv_data[1]

["Harini Komaravelli Test Analyst at Oracle, Hyderabad  Hyderabad, Telangana - Email me on Indeed: indeed.com/r/Harini- Komaravelli/2659eee82e435d1b  ➢ 6 Yrs. of IT Experience in Manual and Automation testing.  WORK EXPERIENCE  QA Analyst  Oracle  Test Analyst at Oracle, Hyderabad  Infosys Ltd -  Hyderabad, Telangana -  November 2011 to February 2016  Hyderabad from Nov 2011 to Feb17 2016 ➢ Worked in Tata Consultancy Services, Hyderabad from Feb 24 to Apr 11 2017 ➢ Currently working as a Test Analyst at Oracle, Hyderabad  QA Analyst with 6 years of IT experience  Oracle  EDUCATION  MCA  Osmania University  B.Sc. in Computer Science  Osmania University  SKILLS  Functional Testing, Blue Prism, Qtp  ADDITIONAL INFORMATION  Area of Expertise:  ➢ Familiar with Agile Methodologies. ➢ Having knowledge in Energy (Petroleum) & Health Care domains. ➢ Involved in preparation of Test Scenarios. ➢ Preparing Test Data for the test cases.  https://www.indeed.com/r/Harini-Komaravelli/2659eee82e435d1b?

In [8]:
def get_spacy_doc(file, data):
    nlp = spacy.blank('en')
    db = DocBin()
        
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']
    
        ents = []
        entity_indices = []
            
        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity == True:
                continue 
                    
            entity_indices = entity_indices + list(range(start, end))  
                
            try:
                span = doc.char_span(start, end,label=label,alignment_mode = 'strict')
            except:
                continue

            if span is None:
                err_data = str([start, end]) + " " + str(text) + "\n "
                file.write(err_data)
            else:
                ents.append(span)

        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass 
    return db

In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [10]:
 len(train),len(test)

(140, 60)

In [11]:
file = open('error.txt','w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 60/60 [00:00<00:00, 77.07it/s]


In [12]:
len(db.tokens)

60

In [13]:
!python -m spacy train /kaggle/working/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-02-08 19:47:42.585522: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 19:47:42.586405: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 19:47:42.587390: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 19:47:42.588172: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 19:47:42.588963: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [14]:
nlp = spacy.load('/kaggle/working/output/model-best')

In [15]:
txt = 'My name is Kashmira Sankhe. I work as a dentist. I completed my degree in Bachelors of Dental Surgery from Government dental College Mumbai. SKILLs : Dental Works'

In [16]:
doc = nlp(txt)
for ent in doc.ents:
    print(ent.label_.upper(),"  =>  ", ent.text)

DEGREE   =>   Bachelors of Dental Surgery
COLLEGE NAME   =>   Government dental College Mumbai


In [17]:
NLP = spacy.load('/kaggle/working/output/model-last')

In [18]:
TXT = "Harini Komaravelli Test Analyst at Oracle, Hyderabad  Hyderabad, Telangana - Email me on Indeed: indeed.com/r/Harini- Komaravelli/2659eee82e435d1b  ➢ 6 Yrs. of IT Experience in Manual and Automation testing.  WORK EXPERIENCE  QA Analyst  Oracle  Test Analyst at Oracle, Hyderabad  Infosys Ltd -  Hyderabad, Telangana -  November 2011 to February 2016  Hyderabad from Nov 2011 to Feb17 2016 ➢ Worked in Tata Consultancy Services, Hyderabad from Feb 24 to Apr 11 2017 ➢ Currently working as a Test Analyst at Oracle, Hyderabad  QA Analyst with 6 years of IT experience  Oracle  EDUCATION  MCA  Osmania University  B.Sc. in Computer Science  Osmania University  SKILLS  Functional Testing, Blue Prism, Qtp  ADDITIONAL INFORMATION  Area of Expertise:  ➢ Familiar with Agile Methodologies. ➢ Having knowledge in Energy (Petroleum) & Health Care domains. ➢ Involved in preparation of Test Scenarios. ➢ Preparing Test Data for the test cases.  https://www.indeed.com/r/Harini-Komaravelli/2659eee82e435d1b?isid=rex-download&ikw=download-top&co=IN https://www.indeed.com/r/Harini-Komaravelli/2659eee82e435d1b?isid=rex-download&ikw=download-top&co=IN   ➢ Experienced in development and execution of Test cases effectively. ➢ Experienced in Functional testing, GUI testing, Smoke testing, Regression testing and Integration Testing ➢ Experienced in doing Accessibility testing of an application ➢ Ability to understand user Requirements, Functional and Design specifications. ➢ Good knowledge of SDLC and STLC processes. ➢ Deciding the Severity and Priority of bugs. ➢ Experience in using Microsoft Test Manager & Oracle Test Manager as Test Management Tools. ➢ Having good experience in testing windows based & web based applications. ➢ Involved in Client Interactions for reviews, issues and for any clarifications. ➢ Web Services Testing ➢ Writing Test Scripts in QTP, Testcomplete. ➢ Creating Object Repositories and Function Libraries in QTP. ➢ Enhanced QTP scripts using VB Script. ➢ Strong experience in working with Blue Prism tool ➢ Worked on different Environments like Windows Application & Web Application  Technical Skills:  ❑ Test Automation Tools: Blue Prism, QTP 10.0, Testcomplete ❑ Test Management Tool: Microsoft Test Manager, Oracle Test Manager & JIRA ❑ Databases: Oracle 10g, SQL Server.  ❑ Operating Systems: Windows 7  Project 1: Title: Cadence Client: Baker Hughes  Technologies: Microsoft Visual Studio and Microsoft Team Foundation Server  Client Background: An oilfield services company delivering focused efforts on shale gas and other oilfield services. It provides services, tools and software for drilling and formation evaluation, well completion, production management, seismic data collection and interpretation.  Project Description: AUT (Application under test) is the next generation revolutionary, robust, easy to use scalable well site data acquisition processing and interpretation system for Client's Drilling Services to deliver services that meets cross divisional business requirements consistently.  Project 2:  Description: Paragon supports your entire care team with one tool that your clinicians need to help deliver the best patient care. Designed by physicians, nurses, pharmacists and mid level providers that have a first-hand understanding of clinical workflow needs, Paragon clinical applications allow your caregivers to focus on what matters most; spending time caring for patients. Since Paragon is fully-integrated across all applications and built around a single patient database, information    entered anywhere in the system is immediately available to the entire care team. Immediate access not only helps clinicians make better treatment decisions - it also helps promote patient safety. Paragon offers a broad suite of multidisciplinary clinical software solutions together with anytime, anywhere access to the complete patient record.  Responsibilities:  • Performed Smoke testing and Regression testing. • Involved in Generating and Executing Test Script using Quick Test Pro & Blue Prism • Usability and User Interface Testing. • Involved in Defect tracking and reporting the bugs using TFS • Participated in frequent walk-through meetings with Internal Quality Assurance groups and with development groups. • Participated in client calls and clarifying the doubts by having AT&T sessions • Involved in functional, regression and smoke testing to validate the application data changes done in windows application • Certifying the build status by running the scripts as part of smoke testing  Project 3:  Description: Food & Beverages R&A: Easily manage business across multiple locations while reducing IT cost and complexity. Cloud-based point-of-sale (POS) solutions enable centralized enterprise management with lower upfront costs and a smaller footprint.  Responsibilities:  • Performed Functional testing and Regression testing. • Involved in Generating and Executing Test Scripts using Blue Prism tool and Open script • Involved in preparing bots using Blue Prism tool. • Accessibility testing of the web application • Involved in Defect tracking and reporting the bugs using JIRA • WebServices testing by calling API's to export the data"

In [19]:
doc = NLP(TXT)
for ent in doc.ents:
    print(ent.label_.upper(),"  =>  ", ent.text)

NAME   =>   Harini Komaravelli
DESIGNATION   =>   Test Analyst
COMPANIES WORKED AT   =>   Oracle
LOCATION   =>   Hyderabad
LOCATION   =>   Hyderabad
YEARS OF EXPERIENCE   =>   6 Yrs
DESIGNATION   =>   Manual and Automation testing.
DESIGNATION   =>   QA Analyst
COMPANIES WORKED AT   =>   Oracle
DESIGNATION   =>   Test Analyst
COMPANIES WORKED AT   =>   Oracle
LOCATION   =>   Hyderabad
LOCATION   =>   Hyderabad
LOCATION   =>   Hyderabad
LOCATION   =>   Hyderabad
DESIGNATION   =>   Test Analyst
COMPANIES WORKED AT   =>   Oracle
LOCATION   =>   Hyderabad
DESIGNATION   =>   QA Analyst
COMPANIES WORKED AT   =>   Oracle
DEGREE   =>   MCA
COLLEGE NAME   =>   Osmania University
DEGREE   =>   B.Sc. in Computer Science
COLLEGE NAME   =>   Osmania University
SKILLS   =>   Functional Testing, Blue Prism, Qtp
COMPANIES WORKED AT   =>   Oracle
COMPANIES WORKED AT   =>   Oracle
COMPANIES WORKED AT   =>   Oracle


In [20]:
!pip install pyMuPDF

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 55.2 MB/s eta 0:00:00


In [21]:
import sys, fitz

In [22]:
fname = '/kaggle/input/pdftest/software-tester-1565000935.pdf'
doc = fitz.open(fname)

In [23]:
text = " "
for page in doc:
    text = text + str(page.get_text())

In [24]:
text = text.strip()

In [25]:
text = ' '.join(text.split())

In [26]:
text

'Software Tester ROBERT SMITH Phone: (123) 456 78 99 Email: info@qwikresume.com Website: www.qwikresume.com LinkedIn: linkedin.com/qwikresume Address: 1737 Marshville Road, Alabama. Objective Around 7 years of software testing experience with thorough knowledge of project & software life cycle methodologies. Led a testing team of 23 members which involved teams from both offshore and onshore. Developed and implemented numerous testing guidelines and principles which have led to successful collaborative projects. An accomplished software tester with professionalism and enthusiasm. Skills SDLC Customer Service Scripting & Programming. Work Experience Software Tester ABC Corporation \xad September 2007 – May 2013 \uf09f Maintained all the business requirements in Quality Center. \uf09f Managed, documented and controlled all the test scenarios, test cases, and expected results in Quality Center. \uf09f Tracked change request activities associated with software development, including enhanc

In [27]:
doc = NLP(text)
for ent in doc.ents:
    print(ent.label_.upper(),"  =>  ", ent.text)

DESIGNATION   =>   Software Tester ROBERT SMITH
DEGREE   =>   BS in Mathematics
COLLEGE NAME   =>   Osmania University


In [28]:
config = nlp.config
bytes_data = nlp.to_bytes()

In [29]:
lang_cls = spacy.util.get_lang_class(config["nlp"]["lang"])
nlpmodel = lang_cls.from_config(config)
nlpmodel.from_bytes(bytes_data)